In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="gemma:7b")

prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")


chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic": "deep learning"}))


In [ ]:
topic = {"topic": "Covid 19"} 


for chunks in chain.stream(topic):
    print(chunks, end="", flush=True)


In [ ]:
topic = {"topic": "LangChain"}  

async for chunks in chain.astream(
    topic
):  
    print(chunks, end="", flush=True)  


In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="gemma:7b", 
    format="json", 
    temperature=0,
)


In [ ]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
     
        content="Tell me 10 places to travel in Europe. resonse in JSON format."
    )
]

chat_model_response = llm.invoke(messages) 
print(chat_model_response.content) 


In [ ]:
import json

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "occupation": {
            "title": "Occupation",
            "description": "The person's Occupation",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

llm = ChatOllama(model="gemma:7b") 

messages = [
    HumanMessage(
    
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{dumps}"),
    HumanMessage(
       
        content="""Now, considering the schema, please describe following person:
        Her name is Eun-Chae Lee, she is 25 years old, and she is a software engineer.
        """
    ),
]

prompt = ChatPromptTemplate.from_messages(
    messages
)  

dumps = json.dumps(json_schema, indent=2)

chain = (
    prompt | llm | StrOutputParser()
)  

print(chain.invoke({"dumps": dumps}))  


In [ ]:
%pip install --upgrade --quiet  pillow


In [ ]:
import base64
from io import BytesIO

from IPython.display import HTML, display
from PIL import Image


def convert_to_base64(pil_image):
    """
    PIL 이미지를 Base64로 인코딩된 문자열로 변환합니다.

    :param pil_image: PIL 이미지
    :return: 크기 조정된 Base64 문자열
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG") 
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str


def plt_img_base64(img_base64):
    """
    Base64로 인코딩된 문자열을 이미지로 표시합니다.

    :param img_base64:  Base64 문자열
    """
   
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    display(HTML(image_html))


file_path = "./images/jeju-beach.jpg"
pil_image = Image.open(file_path)

image_b64 = convert_to_base64(pil_image)
plt_img_base64(image_b64)


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

llm = ChatOllama(model="llava:7b", temperature=0)


def prompt_func(data): 
    text = data["text"] 
    image = data["image"]  

    image_part = { 
        "type": "image_url", 
        "image_url": f"data:image/jpeg;base64,{image}", 
    }

    content_parts = [] 

    text_part = {"type": "text", "text": text} 

    content_parts.append(image_part)
    content_parts.append(text_part)  

    return [HumanMessage(content=content_parts)] 

chain = prompt_func | llm | StrOutputParser()

query_chain = chain.invoke( 
    {"text": "Describe a picture in bullet points.", "image": image_b64}
)

print(query_chain) 
